<a href="https://colab.research.google.com/github/debashisdotchatterjee/Mineral-11/blob/main/Mineral_paper_Objective_1_(afresh)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================
# Mineral Symmetry × Chemistry — Colab Script
# Works directly with KaggleHub (or local upload fallback)
# Builds groups from chemistry and reproduces colorful figures/tables
# ============================================

# -- Installs --
!pip -q install kagglehub pandas numpy scipy matplotlib

# -- Imports --
import os, re, json, glob, zipfile, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import kagglehub

warnings.filterwarnings("ignore")
plt.rcParams.update({
    "figure.dpi": 150, "savefig.dpi": 300,
    "font.size": 11, "axes.titlesize": 12, "axes.labelsize": 11,
    "legend.fontsize": 10, "xtick.labelsize": 10, "ytick.labelsize": 10
})

# -- Output dirs --
ROOT = Path("/content")
OUT  = ROOT / "outputs"
FIG  = OUT / "figures"
TAB  = OUT / "tables"
DATA = OUT / "data"
for p in (OUT, FIG, TAB, DATA): p.mkdir(parents=True, exist_ok=True)

# -- Get dataset (KaggleHub, then CLI fallback, then manual upload) --
def find_master_csv(folder: Path):
    csvs = sorted(folder.rglob("*.csv"), key=lambda p: p.stat().st_size, reverse=True)
    for p in csvs:
        if "Minerals_Database" in p.name or p.name.lower().startswith("minerals"):
            return p
    return csvs[0] if csvs else None

csv_path = None
try:
    print("Attempting KaggleHub download...")
    dpath = Path(kagglehub.dataset_download("vinven7/comprehensive-database-of-minerals"))
    m = find_master_csv(dpath)
    if m is not None:
        csv_path = m
        print("Using:", csv_path)
except Exception as e:
    print("KaggleHub failed:", e)

if csv_path is None:
    try:
        print("Trying Kaggle CLI...")
        !pip -q install kaggle
        if os.path.exists("kaggle.json"):
            os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
            !mv kaggle.json ~/.kaggle/
            !chmod 600 ~/.kaggle/kaggle.json
        dl = ROOT / "kaggle_dl"
        dl.mkdir(exist_ok=True)
        !kaggle datasets download -d vinven7/comprehensive-database-of-minerals -p $dl -q --unzip
        csv_path = find_master_csv(dl)
        print("Using:", csv_path)
    except Exception as e:
        print("Kaggle CLI failed:", e)

if csv_path is None:
    print("Upload your CSV (e.g., Minerals_Database.csv) via the Colab file browser, then set LOCAL_CSV below.")
    LOCAL_CSV = "Minerals_Database.csv"  # <- change if needed
    assert os.path.exists(LOCAL_CSV), "Please upload Minerals_Database.csv then set LOCAL_CSV."
    csv_path = Path(LOCAL_CSV)

# -- Helpers --
def parse_numlike(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    if not s: return np.nan
    s = s.replace("to","-").replace("–","-").replace("—","-").replace("±"," ")
    nums = re.findall(r"[-+]?\d*\.?\d+(?:[Ee][-+]?\d+)?", s)
    if not nums: return np.nan
    vals = [float(t) for t in nums]
    return float(np.mean(vals))

def palette(names):
    base = ["#4E79A7","#F28E2B","#E15759","#76B7B2","#59A14F","#EDC948","#B07AA1","#FF9DA7"]
    return {n: base[i % len(base)] for i, n in enumerate(names)}

def savefig(fig, name):
    pdf, png = FIG/f"{name}.pdf", FIG/f"{name}.png"
    fig.savefig(pdf, bbox_inches="tight"); fig.savefig(png, bbox_inches="tight"); plt.close(fig)

def save_csv(df_, name): df_.to_csv(TAB/name, index=False)
def save_latex(df_, caption, label, name, float_cols=None, fmt="{:.2f}"):
    d2 = df_.copy()
    if float_cols:
        for c in float_cols:
            if c in d2.columns:
                d2[c] = d2[c].map(lambda x: fmt.format(x) if pd.notna(x) else "")
    with open(TAB/name, "w") as f:
        f.write(d2.to_latex(index=False, escape=True, caption=caption, label=label))

# -- Load CSV --
raw = pd.read_csv(csv_path, low_memory=False)

# Column handles
def col(candidates):
    for c in candidates:
        if c in raw.columns: return c
    return None

c_name = col(["Name","name"])
c_sys  = col(["Crystal Structure","crystal_structure","Crystal structure"])
c_hard = col(["Mohs Hardness","mohs_hardness","Mohs hardness"])
c_sg   = col(["Specific Gravity","specific_gravity","Density","density"])
c_ri   = col(["Refractive Index","refractive_index"])

# Build working frame
w = pd.DataFrame({
    "name": raw[c_name],
    "system_code": raw[c_sys],
    "mohs": raw[c_hard].map(parse_numlike) if c_hard else np.nan,
    "sg": raw[c_sg].map(parse_numlike) if c_sg else np.nan,
    "ri": raw[c_ri].map(parse_numlike) if c_ri else np.nan,
})

# anions (present in this dataset)
for an in ["Carbonate","Phosphate","Sulphate","Cyanide","Nitrate","Hydroxyl","Acetate","Ammonium","Hydrated Water"]:
    if an in raw.columns:
        w[an.lower()] = raw[an].fillna(0)
    else:
        w[an.lower()] = 0

# elements
for el in ["Oxygen","Silicon","Fluorine","Chlorine","Bromine","Iodine"]:
    if el in raw.columns:
        w[el.lower()] = raw[el].fillna(0)
    else:
        w[el.lower()] = 0

# Crystal system mapping (inferred from known minerals)
code_map = {0.0:"Unknown",0:"Unknown",1.0:"Triclinic",1:"Triclinic",2.0:"Monoclinic",2:"Monoclinic",
            3.0:"Orthorhombic",3:"Orthorhombic",4.0:"Tetragonal",4:"Tetragonal",
            5.0:"Hex/Trig",5:"Hex/Trig",6.0:"Cubic",6:"Cubic"}
w["system"] = w["system_code"].map(code_map)

# Chemical grouping
def classify_group(r):
    if r.get("carbonate",0) > 0: return "Carbonate"
    if r.get("phosphate",0) > 0: return "Phosphate"
    if r.get("sulphate",0)  > 0: return "Sulfate"
    hal = r.get("fluorine",0)+r.get("chlorine",0)+r.get("bromine",0)+r.get("iodine",0)
    if hal > 0 and r.get("oxygen",0) <= 1: return "Halide"
    if r.get("silicon",0) > 0 and r.get("oxygen",0) > 0: return "Silicate"
    if r.get("oxygen",0) > 0: return "Oxide"
    return "Other"
w["group"] = w.apply(classify_group, axis=1)

# Keep known systems
w_known = w[w["system"].notna() & (w["system"]!="Unknown")].copy()
w.to_csv(DATA/"cleaned_working_set.csv", index=False)

# Orders & palettes
order_sys = ["Cubic","Hex/Trig","Tetragonal","Orthorhombic","Monoclinic","Triclinic"]
order_grp = ["Silicate","Oxide","Carbonate","Phosphate","Sulfate","Halide","Other"]
SYS_COL = palette(order_sys)
GRP_COL = palette(order_grp)

# Contingency
cont = (w_known.assign(cnt=1)
        .pivot_table(index="group", columns="system", values="cnt", aggfunc="sum", fill_value=0)
        .reindex(index=order_grp, columns=order_sys, fill_value=0))
save_csv(cont.reset_index(), "contingency_counts.csv")

# Dirichlet enrichment
global_counts = cont.sum(axis=0).values.astype(float)
p_global = (global_counts + 1e-12) / (global_counts.sum() + 1e-12)
tau = 6.0; alpha = tau * p_global
global_post = (alpha + global_counts) / (alpha.sum() + global_counts.sum())
rng = np.random.default_rng(2025); NS = 20000

def dirichlet_samples(a, size): return rng.dirichlet(a, size=size)
rows, ES_mat, ES_lo, ES_hi = [], pd.DataFrame(index=cont.index, columns=cont.columns), None, None
ES_lo = ES_mat.copy(); ES_hi = ES_mat.copy()

for g in cont.index:
    ng = cont.loc[g, order_sys].values.astype(float)
    a_post = alpha + ng
    thetas = dirichlet_samples(a_post, NS)
    mean = thetas.mean(axis=0)
    lo = np.quantile(thetas, 0.025, axis=0)
    hi = np.quantile(thetas, 0.975, axis=0)
    ES = mean / global_post
    ES_mat.loc[g, order_sys] = ES
    ES_lo.loc[g, order_sys]  = lo / global_post
    ES_hi.loc[g, order_sys]  = hi / global_post
    for s, m, l, h in zip(order_sys, ES, lo/global_post, hi/global_post):
        rows.append({"group":g,"system":s,"ES":float(m),"ES_lo":float(l),"ES_hi":float(h)})

es_df = pd.DataFrame(rows)
save_csv(es_df, "enrichment_ES_with_HDI.csv")

# MI and groupwise KL
N = cont.values.sum()
p_gc = cont.values / N if N>0 else np.zeros_like(cont.values, dtype=float)
p_g  = p_gc.sum(axis=1, keepdims=True); p_c = p_gc.sum(axis=0, keepdims=True)
with np.errstate(divide='ignore', invalid='ignore'):
    mi = float(np.nansum(p_gc * (np.log(p_gc + 1e-12) - np.log(p_g + 1e-12) - np.log(p_c + 1e-12))))
pc = (global_counts + 1e-12) / (global_counts.sum() + 1e-12)
kl_rows=[]
for g in cont.index:
    gc = cont.loc[g, order_sys].values.astype(float)
    pcg = (gc + 1e-12)/(gc.sum()+1e-12)
    kl_rows.append({"group":g, "KL_gc":float(np.sum(pcg*np.log(pcg/pc)))})
kl_df = pd.DataFrame(kl_rows); save_csv(kl_df, "groupwise_KL.csv")

# Densest minerals per system
dense = w_known.dropna(subset=["sg"])
topN=[]
for s in order_sys:
    t = (dense[dense["system"]==s]
         .sort_values("sg", ascending=False)
         .loc[:, ["name","system","sg","group"]].head(5).copy())
    t["rank"]=range(1, len(t)+1); topN.append(t)
densest = pd.concat(topN, ignore_index=True)
save_csv(densest, "densest_by_system.csv")
# LaTeX
with open(TAB/"densest_by_system.tex","w") as f:
    f.write(densest.to_latex(index=False, escape=True, caption="Five densest minerals per crystal system.", label="tab:densest"))

# ----- FIGURES (matplotlib only; colorful) -----
# 1) Global crystal-system distribution
counts_sys = cont.sum(axis=0).reindex(order_sys)
fig = plt.figure(figsize=(8,4.5)); ax = fig.add_subplot(111)
bars = ax.bar(counts_sys.index, counts_sys.values, color=[SYS_COL[s] for s in order_sys], edgecolor="black")
ax.set_xlabel("Crystal system"); ax.set_ylabel("Count"); ax.set_title("Global crystal-system distribution (known systems)")
for b in bars: ax.text(b.get_x()+b.get_width()/2, b.get_height(), f"{int(b.get_height())}", ha="center", va="bottom", fontsize=9)
savefig(fig, "fig_global_crystal_distribution")

# 2) Group-wise spectra
for g in order_grp:
    if g not in cont.index: continue
    vals = cont.loc[g, order_sys]
    fig = plt.figure(figsize=(8,4.2)); ax = fig.add_subplot(111)
    bars = ax.bar(order_sys, vals.values, color=GRP_COL[g], edgecolor="black", alpha=0.9)
    ax.set_xlabel("Crystal system"); ax.set_ylabel("Count"); ax.set_title(f"Crystal-system spectrum: {g}")
    for b in bars: ax.text(b.get_x()+b.get_width()/2, b.get_height(), f"{int(b.get_height())}", ha="center", va="bottom", fontsize=9)
    savefig(fig, f"fig_spectrum_{g.lower()}".replace(" ","_"))

# 3) Mohs hardness violin by system (custom violin without seaborn)
hard_ok = w_known.dropna(subset=["mohs"])
fig = plt.figure(figsize=(9,4.2)); ax = fig.add_subplot(111)
positions = np.arange(len(order_sys))+1; width = 0.35
for i, sys in enumerate(order_sys):
    data = hard_ok.loc[hard_ok["system"]==sys, "mohs"].dropna().values
    if len(data) < 10: continue
    kde = gaussian_kde(data); xs = np.linspace(max(1,data.min()-0.5), min(10,data.max()+0.5), 200)
    ys = kde(xs); ys = ys/ys.max()*width
    ax.fill_betweenx(xs, i+1-ys, i+1+ys, facecolor=SYS_COL[sys], alpha=0.7, edgecolor="black", linewidth=0.8)
    med = np.median(data); q1,q3 = np.percentile(data,[25,75])
    ax.plot([i+1-0.12, i+1+0.12], [med,med], color="black", linewidth=2)
    ax.plot([i+1, i+1], [q1,q3], color="black", linewidth=2)
ax.set_xticks(positions); ax.set_xticklabels(order_sys)
ax.set_ylabel("Mohs hardness"); ax.set_title("Mohs hardness by crystal system")
savefig(fig, "fig_hardness_violin")

# 4) SG vs RI scatter by group
sg_ri = w_known.dropna(subset=["sg","ri"])
fig = plt.figure(figsize=(7.5,5.2)); ax = fig.add_subplot(111)
for g, sub in sg_ri.groupby("group"):
    ax.scatter(sub["ri"], sub["sg"], s=14, alpha=0.75, label=g, color=GRP_COL.get(g,"#333333"), edgecolors="none")
ax.set_xlabel("Refractive index (n)"); ax.set_ylabel("Specific gravity"); ax.set_title("Specific gravity vs refractive index")
ax.legend(title="Group", frameon=True, fontsize=9)
savefig(fig, "fig_sg_vs_ri_scatter")

# 5) Enrichment heatmap (ES)
hm = ES_mat.reindex(index=order_grp, columns=order_sys).astype(float)
fig = plt.figure(figsize=(8.5,3.8)); ax = fig.add_subplot(111)
cmap = plt.get_cmap("coolwarm")
im = ax.imshow(hm.values, cmap=cmap, vmin=0.6, vmax=max(2.0, float(np.nanmax(hm.values))), aspect="auto")
ax.set_xticks(np.arange(len(order_sys))); ax.set_xticklabels(order_sys)
ax.set_yticks(np.arange(len(order_grp))); ax.set_yticklabels(order_grp)
for i in range(hm.shape[0]):
    for j in range(hm.shape[1]):
        val = hm.values[i,j]; ax.text(j,i, "" if np.isnan(val) else f"{val:.2f}", ha="center", va="center", fontsize=9, color="black")
ax.set_title("Posterior enrichment ES (Dirichlet–multinomial; center≈1)")
cbar = fig.colorbar(im, ax=ax, shrink=0.9); cbar.set_label("Enrichment ES")
savefig(fig, "fig_enrichment_heatmap")

# 6) Enrichment profile per group
for g in order_grp:
    if g not in ES_mat.index: continue
    row = es_df[es_df["group"]==g].set_index("system").reindex(order_sys)
    fig = plt.figure(figsize=(8.2,4.0)); ax = fig.add_subplot(111)
    ax.axhline(1.0, color="#666", linestyle="--", linewidth=1)
    y = row["ES"].astype(float).values; lo = row["ES_lo"].astype(float).values; hi = row["ES_hi"].astype(float).values
    x = np.arange(len(order_sys)); yerr = np.vstack([y-lo, hi-y])
    ax.errorbar(x, y, yerr=yerr, fmt="o", capsize=4, linewidth=1.6, markersize=6, color=GRP_COL.get(g,"#333"))
    ax.set_xticks(x); ax.set_xticklabels(order_sys); ax.set_ylabel("Enrichment ES"); ax.set_xlabel("Crystal system")
    ax.set_ylim(0, max(2.5, float(np.nanmax(hi))+0.2)); ax.set_title(f"Posterior enrichment profile: {g}")
    savefig(fig, f"fig_es_{g.lower()}".replace(" ","_"))

# Summary JSON
summary = {
    "n_total": int(len(raw)),
    "n_with_known_system": int(len(w_known)),
    "systems_total_counts": {s:int(c) for s,c in zip(order_sys, cont.sum(axis=0).reindex(order_sys).values)},
    "mutual_info_nats": float(mi),
}
with open(DATA/"results_summary.json","w") as f: json.dump(summary, f, indent=2)

# One-click bundle
ZIP = ROOT/"mineral_outputs.zip"
with zipfile.ZipFile(ZIP, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for folder in [FIG, TAB, DATA]:
        for p in Path(folder).rglob("*"):
            zf.write(p, arcname=str(p.relative_to(OUT.parent)))
print("All done! Download:", ZIP)


Attempting KaggleHub download...
Using: /kaggle/input/comprehensive-database-of-minerals/Minerals_Database.csv
All done! Download: /content/mineral_outputs.zip
